In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from buck.analysis.basics import ingest_images

# Your existing ingestion
fpath = "..\\images\\squared\\*_NDA.png"
images, ages = ingest_images(fpath)

from buck.analysis.basics import split_data

Xtr_og, ytr_og, Xval, yval, Xte, yte_onehot, ages, l_map = split_data(images, ages)

(143, 288, 288, 1) (36, 288, 288, 1) (45, 288, 288, 1)


In [2]:
from buck.analysis.basics import homogenize_data

augment_multiplier = 40
X_train_flat, y_train_flat, X_test_flat, y_true, label_mapping, num_classes = homogenize_data(Xtr_og, ytr_og, Xte,yte_onehot, l_map, augment_multiplier)

  Class 0 (Age 1.5): 20 → 1360 samples
  Class 1 (Age 2.5): 25 → 1360 samples
  Class 2 (Age 3.5): 30 → 1360 samples
  Class 3 (Age 4.5): 34 → 1360 samples
  Class 4 (Age 5.5): 34 → 1360 samples


In [3]:
from buck.classifiers.autotune import optimize_all

optimize_all(X_train_flat, y_train_flat, X_test_flat, y_true, cycles=1)

Optimizing

...bagging classifier



KeyboardInterrupt



In [ ]:
# Random Forest
from buck.classifiers.random_forest import (
    _optimize_rs, _optimize_nest, _optimize_max_d, _optimize_crit, _optimize_cw, _optimize_mss, _optimize_msl, _optimize_mwfl, _optimize_mf, _optimize_mln, _optimize_mid
)

# Shorten parameters
Xtr_pca = X_train_flat
ytr_flat = y_train_flat
Xte_pca = X_test_flat

opts = {
    "n_estimators": 100,
    "criterion": "gini",
    "max_depth": None,
    "min_samples_split": 2,
    "min_samples_leaf": 1,
    "min_weight_fraction_leaf": 0.0,
    "max_features": "sqrt",
    "max_leaf_nodes": None,
    "min_impurity_decrease": 0.0,
    "bootstrap": True,
    "oob_score": False,
    "n_jobs": -1,
    "random_state": 42,
    "verbose": 0,
    "warm_start": False,
    "class_weight": None,
    "ccp_alpha": 0.0,
    "max_samples": None,
    "monotonic_cst": None,
}

# Optimize hyperparameters
ma_vec = []
f1_vec = []
opts, ma, f1 = _optimize_rs(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
print(ma)
opts, ma, f1 = _optimize_nest(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
print(ma)
opts, ma, f1 = _optimize_max_d(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
print(ma)
opts, ma, f1 = _optimize_crit(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)  # type: ignore
print(ma)
opts, ma, f1 = _optimize_cw(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
print(ma)
opts, ma, f1 = _optimize_mss(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
print(ma)
opts, ma, f1 = _optimize_msl(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
print(ma)
opts, ma, f1 = _optimize_mwfl(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
print(ma)
opts, ma, f1 = _optimize_mf(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
print(ma)
opts, ma, f1 = _optimize_mln(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
print(ma)
opts, ma, f1 = _optimize_mid(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
print(ma)
ma_vec.append(ma)
f1_vec.append(f1)
print(ma, f1)